In [5]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import csv
import os
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import math

In [7]:
'''패치 일자'''
kst = timezone(timedelta(hours=9))
startTime_25_1 = datetime(2025, 1, 9, 7, 0, 0, tzinfo=kst)
startTime_25_5 = datetime(2025, 3, 5, 3, 30, 0, tzinfo=kst)
startTime_25_9 = datetime(2025, 4, 30, 7, 30, 0, tzinfo=kst)
startTime_25_13 = datetime(2025, 6, 25, 7, 0, 0, tzinfo=kst)
startTime_25_17 = datetime(2025, 8, 27, 8, 30, 0, tzinfo=kst)
startTime_25_21 = datetime(2025, 10, 22, 7, 40, 0, tzinfo=kst)
startTime_26_1 = datetime(2026, 1, 8, 7, 20, 0, tzinfo=kst)
startTime_26_2 = datetime(2026, 2, 4, 6, 0, 0, tzinfo=kst)
# print(int(startTime.timestamp()))

# 액트 기준으로 구분하기
patch_times = {
    "period_1": (startTime_25_1, startTime_25_5), # 25_1 ~ 25_5 전
    "period_2": (startTime_25_5, startTime_25_9), # 25_5 ~ 25_9 전
    "period_3": (startTime_25_9, startTime_25_13), # 25_9 ~ 26_13 전
    "period_4": (startTime_25_13, startTime_25_17), # 25_13 ~ 25_17 전
    "period_5": (startTime_25_17, startTime_25_21), # 25_17 ~ 25_21 전
    "period_6": (startTime_25_21, startTime_26_1), # 25_21 ~ 26_1 전
    "period_7": (startTime_26_1, startTime_26_2) # 26_1 ~ 26_2
}

In [ ]:
# Riot API 설정
API_KEY = ""
HEADERS = {"X-Riot-Token": API_KEY}

REGION_MAPPING = {
    'KR': 'asia', 'JP1': 'asia',
    'BR1': 'americas', 'NA1': 'americas', 'LA1': 'americas', 'LA2': 'americas',
    'EUN1': 'europe', 'EUW1': 'europe', 'TR1': 'europe', 'RU': 'europe', 'ME1': 'europe',
    'OC1': 'sea', 'SG2': 'sea', 'TW2': 'sea', 'VN2': 'sea'
}

OUTPUT_FILE = r"F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\matchid_data\collected_match_ids.csv"
PROGRESS_FILE = r"F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\matchid_data\processed_puuids.txt"
SAVE_INTERVAL = 10

def load_processed_puuids():
    # 이미 수집 완료된 유저 PUUID 목록을 불러오는 함수
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return set(line.strip() for line in f)
    return set()

def save_progress(puuids):
    # 새로 수집 완료된 유저 PUUID를 progress 파일에 저장하는 함수
    os.makedirs(os.path.dirname(PROGRESS_FILE), exist_ok=True)
    with open(PROGRESS_FILE, 'a') as f:
        for puuid in puuids:
            f.write(f"{puuid}\n")

def save_to_csv_with_period(data, file_name):
    # 매치 ID와 period 정보를 CSV에 저장하는 함수 (중복 저장 방지용)
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    df = pd.DataFrame(data, columns=['match_id','period'])
    if not os.path.exists(file_name):
        df.to_csv(file_name, index=False, mode='w', encoding='utf-8')
    else:
        df.to_csv(file_name, index=False, mode='a', header=False, encoding='utf-8')

def final_cleanup_by_period(file_name):
    # 저장된 CSV에서 중복 매치 ID를 제거하고 최종 정리하는 함수
    if not os.path.exists(file_name): return
    df = pd.read_csv(file_name)
    before = len(df)
    df.drop_duplicates(subset=['match_id'], inplace=True)
    df.to_csv(file_name, index=False)
    print(f"✅ 최종 정리 완료: {before} -> {len(df)} 고유 매치")

# %% Core: get matches per user
def get_matches_for_one_user(puuid, platform, max_matches=1000):
    # 특정 유저(PUUID)와 서버(platform)에 대해 매치 ID를 최대 max_matches까지 가져오는 함수
    routing = REGION_MAPPING.get(platform.upper(), 'asia')
    url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    all_matches = []
    
    for start_idx in range(0, max_matches, 100):
        # 100개씩 반복 호출
        params = {"start": start_idx, "count": 100, "type":"ranked"}
        while True:
            try:
                time.sleep(0.05)  # API 속도 제한 준수
                r = requests.get(url, headers=HEADERS, params=params)
                
                if r.status_code == 200:
                    fetched = r.json()
                    all_matches.extend(fetched)
                    break
                elif r.status_code == 429:
                    # Rate Limit 초과 시 Retry-After 대기
                    retry = int(r.headers.get("Retry-After",10))
                    time.sleep(retry+1)
                elif r.status_code == 502:
                    # 서버 502 오류 발생 시 잠시 대기 후 재시도
                    time.sleep(1.2)
                else:
                    print(f"Error {r.status_code} for {puuid}")
                    break
            except requests.exceptions.RequestException:
                # 일시적 네트워크 오류 시 1초 대기 후 재시도
                time.sleep(1)
        if len(fetched) < 100:
            # 가져온 데이터가 100개 미만이면 더 이상 호출하지 않음
            break
    return all_matches

# %% Classify period based on timestamp
def classify_match_period(match_ids, platform):
    # 매치 ID 리스트를 받아 각 매치의 timestamp 기준으로 period를 분류하는 함수
    routing = REGION_MAPPING.get(platform.upper(), 'asia')
    results = []
    
    for m_id in match_ids:
        while True:
            try:
                url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/{m_id}"
                r = requests.get(url, headers=HEADERS)
                if r.status_code == 200:
                    data = r.json()
                    ts = data['info']['gameStartTimestamp'] // 1000
                    dt = datetime.fromtimestamp(ts, tz=timezone(timedelta(hours=9)))
                    # 매치 timestamp 기준으로 period 지정
                    for p_name, (s,e) in patch_times.items():
                        if s <= dt < e:
                            results.append((m_id,p_name))
                            break
                    break
                elif r.status_code == 429:
                    retry = int(r.headers.get("Retry-After",10))
                    time.sleep(retry+1)
                elif r.status_code == 502:
                    time.sleep(1.2)
                else:
                    print(f"Failed match {m_id} ({r.status_code})")
                    break
            except requests.exceptions.RequestException:
                time.sleep(1)
    return results

# %% Main collection function
def collect_by_patch_periods(df_dict):
    # 서버별 유저 데이터(df_dict)를 받아 매치 ID 수집, period 분류, CSV 저장까지 수행하는 메인 함수
    processed_puuids = load_processed_puuids()
    session_data = []
    newly_processed = []
    
    for platform, df in df_dict.items():
        total = len(df)
        start_time = time.time()
        print(f"\n🚀 {platform} 지역 수집 시작... (총 {total}명)")
        
        for idx, puuid in enumerate(df['puuid'],1):
            if puuid in processed_puuids: continue
            m_ids = get_matches_for_one_user(puuid, platform)
            classified = classify_match_period(m_ids, platform)
            session_data.extend(classified)
            newly_processed.append(puuid)
            
            # ETA 계산
            elapsed = time.time() - start_time
            eta = (elapsed / idx) * (total - idx)
            eta_str = f"{int(eta//3600):02d}:{int((eta%3600)//60):02d}:{int(eta%60):02d}"
            
            # 진행률 출력
            print(f"[{idx}/{total}] 진행 중.. 저장 완료 (이번 턴 매치: {len(classified)}) | ETA: {eta_str}", end='\r')
            
            if len(newly_processed) >= SAVE_INTERVAL:
                # 중간 저장
                save_to_csv_with_period(session_data, OUTPUT_FILE)
                save_progress(newly_processed)
                session_data.clear()
                processed_puuids.update(newly_processed)
                newly_processed = []
    
    # 남은 데이터 저장
    if newly_processed or session_data:
        save_to_csv_with_period(session_data, OUTPUT_FILE)
        save_progress(newly_processed)
    
    final_cleanup_by_period(OUTPUT_FILE)
    print("\n✨ 모든 지역 수집 완료!")

In [9]:
def unix_conv(unix_time):
    dt = datetime.datetime.utcfromtimestamp(unix_time)
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt

- 함수 모은거

## 샘플 유저 데이터 서버마다 합치기

In [7]:
import warnings
warnings.filterwarnings('ignore')

def perform_stratified_sampling(sample_fraction, input_file, output_file, min_per_group=1, stratify_col='tier', random_state=42):
    """
    계층적 샘플링 수행 (기본: 티어별 샘플링).
    - sample_fraction: 각 그룹에서 비율로 뽑을 비율
    - min_per_group: 그룹이 작을 때 최소 보장 샘플 수
    - stratify_col: 계층화에 사용할 컬럼 이름
    """
    print('📂 데이터 로딩 중...')
    df = pd.read_csv(input_file)

    if stratify_col not in df.columns:
        raise ValueError(f"'{stratify_col}' 컬럼이 입력 파일에 없습니다.")

    # 그룹별로 샘플 수 계산
    def _sample_group(g):
        grp_size = len(g)
        n = max(0, int(round(grp_size * sample_fraction)))
        if n < min_per_group and grp_size > 0:
            n = min(min_per_group, grp_size)
        if n <= 0:
            return g.iloc[0:0]
        return g.sample(n=n, random_state=random_state)

    sampled_df = df.groupby(stratify_col, group_keys=False).apply(_sample_group).reset_index(drop=True)

    # 결과 출력 및 저장
    print('\n✅ 샘플링 완료!')
    print(f"전체 데이터 수: {len(df):,} 명")
    print(f"샘플링 데이터 수: {len(sampled_df):,} 명")
    print(f"샘플 내 {stratify_col} 비중:\n{(sampled_df[stratify_col].value_counts(normalize=True) * 100).round(2)}%")

    sampled_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"💾 샘플 데이터가 {output_file}에 저장되었습니다.")

In [3]:
# 환경에 맞게 파일 경로 입력
file_paths = [
    r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\na1_puuids_high_tiers',
    r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\na1_puuids_iron_diamond',
    r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\eun1_puuids_high_tiers',
    r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\eun1_puuids_iron_diamond'
]

for name in file_paths:
    sampling = 0.002
    perform_stratified_sampling(
        sample_fraction = sampling,
        input_file=f'{name}.csv', 
        output_file=f'{name}_{sampling*100}%.csv'
    )

NameError: name 'perform_stratified_sampling' is not defined

In [10]:
# 서버별 유저 CSV 로드
na1_high=pd.read_csv(r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\0.2%_na1_puuids_high_tiers.csv')
na1_low=pd.read_csv(r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\0.2%_na1_puuids_iron_diamond.csv')
eun1_high=pd.read_csv(r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\0.2%_eun1_puuids_high_tiers.csv')
eun1_low=pd.read_csv(r'F:\porti\2025\스파르타자료들\python_study\venv\04_Team8\0.2%_eun1_puuids_iron_diamond.csv')

# 서버별 합치기 & 중복 제거
def prepare_df(low, high):
    combined = pd.concat([low, high], ignore_index=True)
    combined.drop_duplicates(subset=['puuid'], inplace=True)
    return combined

df_dict = {
    "NA1": prepare_df(na1_low, na1_high),
    "EUN1": prepare_df(eun1_low, eun1_high)
}

print(f"✅ {len(df_dict)}개 지역 데이터 준비 완료! {list(df_dict.keys())}")

✅ 2개 지역 데이터 준비 완료! ['NA1', 'EUN1']


## EXE

In [ ]:
collect_by_patch_periods(df_dict)


🚀 NA1 지역 수집 시작... (총 1475명)


In [ ]:
collect_by_patch_periods(df_dict)


🚀 NA1 지역 수집 시작... (총 1475명)
[10/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[20/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[30/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[40/1475] 진행 중.. 저장 완료 (이번 턴 매치: 61개)
[50/1475] 진행 중.. 저장 완료 (이번 턴 매치: 56개)
[60/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[70/1475] 진행 중.. 저장 완료 (이번 턴 매치: 92개)
[80/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[90/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[100/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[110/1475] 진행 중.. 저장 완료 (이번 턴 매치: 47개)
[120/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[130/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[140/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[150/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[160/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[170/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[180/1475] 진행 중.. 저장 완료 (이번 턴 매치: 36개)
[190/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[200/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[210/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[220/1475] 진행 중.. 저장 완료 (이번 턴 매치: 10개)
[230/1475] 진행 중.. 저장 완료 (이번 턴 매치: 14개)
[240/1475] 진행 중.. 저장 완료 (이번 턴 매치: 100개)
[250/1475] 진행 중.. 저장 완료 (이번

In [ ]:
# collect_by_patch_periods(df_dict) 실행 완료 후 실행할 것
# 의도 : period 기준으로 나누고자 함
# 먼저 매치아이디를 수집한 후, 지정된 period 기준대로 나눠 넣는다.
classify_matchids_by_period()

In [29]:
collect_by_patch_periods(df_dict)


🚀 NA1 지역 수집 시작... (총 1475명)
[10/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1595개)
[20/1475] 진행 중.. 저장 완료 (이번 턴 매치: 951개)
[30/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1240개)
[40/1475] 진행 중.. 저장 완료 (이번 턴 매치: 769개)
[50/1475] 진행 중.. 저장 완료 (이번 턴 매치: 2023개)
[60/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1367개)
[70/1475] 진행 중.. 저장 완료 (이번 턴 매치: 844개)
[80/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1138개)
[90/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1407개)
[100/1475] 진행 중.. 저장 완료 (이번 턴 매치: 940개)
[110/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1376개)
[120/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1718개)
[130/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1272개)
[140/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1262개)
[150/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1334개)
[160/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1719개)
[170/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1641개)
[180/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1010개)
[190/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1014개)
[200/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1447개)
[210/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1748개)
[220/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1488개)
[230/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1012개)
[240/1475] 진행 중.. 저장 완료 (이번 턴 매치: 1704개)


## 매치데이터

In [10]:
matchids = pd.read_csv('data/02_match-v5/collected_match_ids.csv')

In [11]:
print(matchids.head())

         match_id    period
0  LA2_1501465065  period_2
1  LA2_1501457348  period_2
2  LA2_1501453622  period_2
3  LA2_1501455030  period_2
4  LA2_1501453109  period_2


In [ ]:
# url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/{matchid}?api_key={API_KEY}"